In [ ]:
import pandas as pd
from shapely.geometry import Point
# import geopandas as gpd
# from geopandas import GeoDataFrame
import gurobipy as gp
import numpy as np

: 

In [4]:
G = 10000000
H = 78.91
alpha = 0.075
hub= 0.6
lamb = 250 * 365
n = 176
p = 74
q = 8
K = 74
uncertainty = 50
inverse_foods = [1/41.3, 1/107.38, 1/37.9961, 1/330.4]

In [5]:
arcDistance = pd.read_csv("./arcDistance.csv")
demand = pd.read_csv("./demand.csv")
supply = pd.read_csv("./supplyT.csv")
weighting = pd.read_csv("./weighting.csv")
inverse = pd.read_csv("./inverse_populations.csv")

FileNotFoundError: [Errno 2] No such file or directory: './supplyT.csv'

In [ ]:
model = gp.Model()

g=37
model = gp.read(f"out{g}.mps")

X = model.addMVar((n, n, p), lb=0)
Y = model.addMVar((n, n, p), lb=0)
Z = model.addMVar((n, n, p), lb=0)
beta = model.addVar(lb=0, vtype=gp.GRB.INTEGER)

objective=0


for i in range(g+1,n):
    if i < 168:
        model.addConstr(( (sum(Y[:,i,:])) == 0 ))
        model.addConstr(( (sum(Z[i,:,:])) == 0 ))
    for k in range(p):
        if i < 8:
            model.addConstr(( sum(Y[:,i+168,k]) == sum(Z[i+168,:,k]) ))
        model.addConstr((sum(X[i,:,k]) + sum(Y[i,:,k]) <= abs(supply.iloc[i,k])))
    for l in range(n):
        if i < 168 & l < 168:
            objective += arcDistance.iloc[i,l] * sum(X[i,l,:])
        elif i < 168 & l >= 168:
            objective += arcDistance.iloc[i,l] * sum(Y[i,l,:]) * hub
            objective += arcDistance.iloc[i,l] * sum(Z[l,i,:])
        for k in range(4):
            model.addConstr((beta >= 
                sum(sum(inverse.iloc[i,0] * (demand.iloc[i,k] - 
                10000000 * np.dot((sum(X[:,i,k]) + sum(Z[:,i,k])), weighting))) *
                np.array([1/41.3, 1/107.38, 1/37.9961, 1/330.4]))))
    model.write(f"out{i}.mps")

objective = objective * alpha * H    
objective += (1-alpha) * beta * mu

model.write("out.mps")

model.setObjective(objective, gp.GRB.MINIMIZE)
model.optimize()

In [ ]:
Xval = X.X
Yval = Y.X
Zval = Z.X
betaVal = beta.X

Xval = np.array(Xval)
Yval = np.array(Yval)
Zval = np.array(Zval)
betaVal = np.array(betaVal)

np.save('Xval.npy', Xval)
np.save('Yval.npy', Yval)
np.save('Zval.npy', Zval)
np.save('betaVal.npy', betaVal)


In [ ]:
df = pd.read_csv("cluster_centers.csv", delimiter=',', skiprows=0, low_memory=False)
print(df)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = GeoDataFrame(df, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15)

In [ ]:
df = pd.read_csv("locations_clustered.csv", delimiter=',', skiprows=0, low_memory=False)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = GeoDataFrame(df, geometry=geometry)

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', c=df['clusters'], markersize=15);